In [ ]:
# some imports
import os
import sys
import glob
import warnings
warnings.filterwarnings("ignore")

from copy import copy

import numpy as np
import torch.nn as nn
from tqdm.notebook import tqdm_notebook

sys.path.append('../..')

from seismiqb import *
from seismiqb.src.controllers.torch_models import ExtensionModel

from seismiqb.batchflow import FilesIndex, Pipeline
from seismiqb.batchflow import D, B, V, P, R, L

# E cube

In [ ]:
cube_path = '/data/seismic_data/seismic_interpretation/CUBE_01_ETP/amplitudes_01_ETP.hdf5'
coherency_path = '/data/seismic_data/seismic_interpretation/CUBE_01_ETP/coherency/E_anon_Var.hdf5'
fault_files = sorted(glob.glob(os.path.join(os.path.dirname(cube_path), 'INPUTS/FAULTS/PROCESSED/*')))

for fault_file in fault_files:
    Fault.check_format(fault_file, verbose=False)

In [ ]:
# ! rm /data/seismic_data/seismic_interpretation/CUBE_24_VZC/INPUTS/FAULTS/PROCESSED/M/Fault_interpretation_15

In [ ]:
# Fault.split_file('/data/seismic_data/seismic_interpretation/CUBE_24_VZC/INPUTS/FAULTS/RAW/L_faults_VZC', '/data/seismic_data/seismic_interpretation/CUBE_24_VZC/INPUTS/FAULTS/PROCESSED/L/')

In [ ]:
dataset = SeismicCubeset(FilesIndex(path=cube_path, no_ext=True))

dataset.load_geometries()

dataset.load(label_dir='/INPUTS/FAULTS/PROCESSED/*', labels_class=Fault, transform=True, verify=True)
dataset.modify_sampler(dst='train_sampler', finish=True)

In [ ]:
cube = SeismicGeometry(cube_path)
coherency = SeismicGeometry(coherency_path)

In [ ]:
dataset.labels[0]

In [ ]:
2

In [ ]:
coherency.show_slide(100)

In [ ]:
! cat /data/seismic_data/seismic_interpretation/CUBE_24_VZC/INPUTS/FAULTS/PROCESSED/M/Fault_interpretation_15

In [ ]:
ls 

In [ ]:
! cat /data/seismic_data/seismic_interpretation/CUBE_23_VYNGAYAHA/INPUTS/FAULTS/PROCESSED/Fault_Patches_3_137

# Map of faults

In [ ]:
_ = dataset.show_slices(src_sampler='train_sampler',
                        normalize=True, shape=(1, 128, 128),
                        adaptive_slices=False,
                        cmap='Reds', interpolation='bilinear',
                        figsize=(8, 6))

# Initial coordinates

In [ ]:
! head "{fault_files[0]}"

# Shifted and interpolated coordinates

Linear tranformation of coordinates:

    array[:, 0] -= self.geometry.ilines_offset
    array[:, 1] -= self.geometry.xlines_offset
    array[:, 2] -= self.geometry.delay
    array[:, 2] /= self.geometry.sample_rate

In [ ]:
dataset.labels[0][0].points

# Examples

We can see shift of the annotation. Why?

In [ ]:
dataset.labels[0][0].points

In [ ]:
dataset.geometries[0].ilines_offset + 1839

In [ ]:
p = dataset.labels[0][0].points
p[p[:, 0] == 1829]

In [ ]:
dataset.show_slide(dataset.labels[0][0].points[0, 0], n_line=1480, figsize=(15,15))

But there is good example: sticks are in the right place.

In [ ]:
BATCH_SIZE = 64
CROP_SHAPE = (1, 256, 256)
NUM_ITERS = 300
N_STEPS = 8
STRIDE = 20

In [ ]:
load = (Pipeline()
        .crop(points=D('train_sampler')(BATCH_SIZE),
              shape=CROP_SHAPE, side_view=False)
        .create_masks(dst='masks', width=5)
        .load_cubes(dst='images'))

In [ ]:
show_pipeline = (load ) << dataset
batch = show_pipeline.next_batch(1)

In [ ]:
batch.plot_components('images', 'masks', idx=0, mode='separate')

In [ ]:
batch.plot_components('images', 'masks', idx=0, mode='overlap')